In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df_train = pd.read_excel("F:/Context_Aware_Data_Cleaning/Arif/UBMEC_BASE/Dataset/UBMEC.xlsx")

In [8]:
df_train.head()

,text,classes
0,এই নাটকটি হচ্ছে ১৯৯৩ সালের। কিন্তু তখন ইউটিউব ...,joy
1,২০২১ সালে যারা “ কোথাও কেউ নেই ” দেখতে আসে - ত...,surprise
2,"বাংলাদেশের ইতিহাসে সর্বশ্রেষ্ঠ নাটক ""কোথাও কেউ...",joy
3,"এতো সুন্দর নাটক আগে কখনো দেখা হয় নাই,2021 সালে...",joy
4,কখনো চিন্তাও করিনি জন্মের ৭-৮ বছর আগের নাটক দে...,surprise


In [9]:
df_train.dropna(inplace=True)

In [15]:
df_train.dtypesa

text       object
classes    object
dtype: object

In [20]:
# Change data type of 'Column1' to string
df_train['text'] = df_train['text'].astype(str)

In [21]:
longest_string = max(df_train["text"], key=len)
print(len(longest_string))

3648


In [22]:
df_train['classes'].value_counts()

joy         3467
sadness     2683
anger       2480
disgust     2079
surprise    1366
fear        1361
Name: classes, dtype: int64

In [25]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_train, test_size=0.2,stratify=df_train['classes'],random_state=42)

In [29]:
X_train  =  train_df['text'].values.tolist()
y_train = train_df['classes'].tolist()

In [30]:
X_test =  test_df['text'].values.tolist()
y_test = test_df['classes'].tolist()

In [ ]:
pip install ktrain

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-multilingual-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500)

In [ ]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

In [ ]:
print(trn)

In [ ]:
import matplotlib.pyplot as plt

# Calculate token lengths for train and test datasets
train_token_lengths = [len(t.split()) for t in X_train]
test_token_lengths = [len(t.split()) for t in X_test]

# Plot token length distribution
plt.figure(figsize=(10, 6))
plt.hist(train_token_lengths, bins=50, alpha=0.5, color='blue', label='Train')
plt.hist(test_token_lengths, bins=50, alpha=0.5, color='orange', label='Test')
plt.xlabel('Token Length')
plt.ylabel('Frequency')
plt.title('Token Length Distribution')
plt.legend()
plt.show()

In [ ]:
model = t.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=8)

In [ ]:
# Adding early stopping callback from TensorFlow
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', patience=5, mode='auto')

In [ ]:
history = learner.fit_onecycle(2e-5, 10,callbacks=[early_stopping])

In [ ]:
# Save the trained model
learner.save_model('/kaggle/output/model.zip')

In [ ]:
classes=t.get_classes()

In [ ]:
cm = learner.validate(class_names=t.get_classes())

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [ ]:
!zip -r mental.zip /kaggle/output/model